In [ ]:
%%HTML
<!-- Make fonts readable at 1024x768 -->
<style>
.rendered_html { font-size:0.7em; }
</style>

In [ ]:
import numpy as np
import scipy.signal
from scipy import fftpack
%matplotlib notebook
import matplotlib.pylab as plt
from matplotlib import animation, patches
from IPython.display import display, Audio, HTML
import soundfile as sf
from style import *

def clean_4_audio(y):
    if np.amax(np.absolute(y)) > 1.:
        return y/np.amax(np.absolute(y))
    else:
        return np.concatenate((y, [1, -1]))

### Universidad Austral de Chile 

## INFO183: Análisis de sistemas lineales

# Unidad 3: Sistemas para el procesamiento de señales

### Dr. Pablo Huijse, phuijse at inf dot uach dot cl 

### <a href="https://github.com/phuijse/UACH-INFO183"> github.com/phuijse/UACH-INFO183 </a>


***
<a id="index"></a>

# Contenidos de la unidad

***

1. [Definición de sistema](#section1)
1. [Sistema FIR](#section2)
1. [Sistemas LTI y filtros](#section3)
1. [Sistema IIR](#section4)
1. [Transformada Z](#section5)

***
<a id="section1"></a>

# Definición de sistema

***

- *Análisis de señales:* El estudio de las señales y sus propiedades en el dominio del tiempo y frecuencia
- *Procesamiento de señales:* El diseño de **sistemas** que procesan **señales de entrada** y producen **señales de salida**
    - Adicionalmente, un sistema puede tener parámetros (entradas númericas o booleanas)
    - Adicionalmente, un sistema puede tener retornos (salidas númericas o booleanas)
    - Sistema sin señal de entrada: Oscilador
    - Sistema sin señal de salida: Detector/clasificador de señal
    - Existen sistemas analógicos (continuos) y digitales (discretos), nos enfocaremos en los últimos
    
   
<center><img src="img/system.png"></center>

- Usaremos $x[n]$ para denotar la señal (discreta) de entrada y $X[k]$ su espectro
- Usaremos $y[n]$ para denotar la señal (discreta) de salida e $Y[k]$ su espectro
 


### Ejemplos

1. Un sistema para reducir el ruido de una EEG

<center><img src="img/system-denoise-eeg.png"></center>

1. Un sistema para mejorar (sharpen) una imagen fuera de foco

<center><img src="img/system-sharpen.jpg"></center>

1. Un sistema para eliminar el eco de un audio

<center><img src="img/system-echo.png"></center>


***

# Sistemas  sin memoria

Los sistemas sin memoria son de forma

$$
y[n] = f(x[n]),
$$

es decir la salida del sistema en un instante dado depende solo de la entrada en ese instante

### Ejemplos


- Sistema amplificador ideal 
$$
y[n] = A x[n], 
$$
donde $A>0$ se llama *ganancia*
    - provoca atenuación de la entrada si $0<A<1$
    - es un sistema identidad si $A=1$

- Sistema con corrupción cuadrática
$$
y_n = x_n + \epsilon x_n^2
$$

- Sistema saturador (clamp)
$$
y_n = \begin{cases} B &x_n > B \\x_n & x_n \in [-B, B]\\ -B & x_n < -B\end{cases}
$$
- Sistema rectificador
$$
y_n = | x_n |
$$

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
Fs = 22050; n = np.arange(0, 2, step=1.0/Fs)
x = 0.2*np.sin(2.0*np.pi*200*n); 
y = np.abs(x) #+ 2*x**2
ax.plot(n, x, label='input'); ax.plot(n, y, label='output'); 
ax.set_xlim([0, 0.1]); plt.legend()
Audio(clean_4_audio(y), rate=Fs)

***

# Sistema Lineal

Propiedades de los sistemas lineales

- **Homogeneidad:** Un cambio en la amplitud de la entrada produce un cambio equivalente en la salida

$$
f(cx[n]) = c f(x[n]) = c y[n]
$$

- **Aditividad:** Señales que se suman en la entrada producen señales que se suman en la salida

$$
f(x_1[n] + x_2[n]) = f(x_1[n]) + f(x_2[n]) = y_1[n] + y_2[n]
$$

    - Las señales pasan por el sistema sin interactuar entre ellas
- Si no se cumple alguna de estas propiedades el sistema es **no lineal**
- ¿Son los sistemas anteriores lineales?



### Otras propiedades de los sistemas lineales


- Una cascada de sistemas lineales forman un sistema lineal equivalente
    - **Conmutatividad:** El orden de los sistemas en la cascada no es relevante
<img src="img/system-conmu.png" width="400px">

- **Principio de superposición**: 
    - Considere una método de descomposición de funciones (Fourier, Taylor, PCA)
    - Si descomponemos una señal en $M$ componentes: $x[n] = x_1[n] + x_2[n] + \ldots +  x_M[n]$
    - Y aplicamos un **sistema lineal** a cada componente $y_j[n] = f(x_j[n])$
    - Podemos recuperar la salida total usando **aditividad** como $y_1[n] + y_2[n] + \ldots +  y_M[n] = y[n]$

<img src="img/system-superpos.png" width="400px">
    


***

# Sistemas con memoria

Un sistema con memoria es aquel cuya salida puede depender de 
- la entrada actual
- las entradas anteriores
- las salidas anteriores

$$
\begin{align}
y[n] = f(x[n], &x[n-1], x[n-2], \ldots, x[0], \\ \nonumber
&y[n-1], y[n-2], \ldots, y[0]) \nonumber
\end{align}
$$

esto también se conoce como **sistema causal**

Un **sistema no-causal** usa entradas futuras ($x[n+1]$, $x[n+2]$, ...) y por ende solo se puede implementar de forma offline (una vez que sea ha observado toda la señal)

### Ejemplos de sistemas lineales con memoria simples

- Sistema con un retardo (delay)

$$
y[n] = x[n-m],
$$
    - depende solo de "una" entrada anterior
    - el valor de m define que tan "antigua" es la entrada pasada

- Sistema reverberador (eco)

$$
y[n] = x[n] + A x[n-m],
$$

    - depende de una entrada "pasada" y la entrada actual
    - la ganancia controla si el "eco" se atenua o amplifica

- El delay no afecta la amplitud de los componentes frecuenciales pero si su fase
- Más adelante veremos que este es un tipo de filtro conocido como pasa-todo (*all-pass*)

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(6, 6))
n = np.arange(0, 400, step=1)
x = lambda m: np.sin(2.0*np.pi*0.05*(n-m)) 
f = fftpack.fftshift(fftpack.fftfreq(d=1, n=len(n)))

def update(m):
    m = m*0.5 # Para hacer la animación + fluida
    ax[0].cla(); ax[0].plot(n, x(m));
    X = fftpack.fftshift(fftpack.fft(x(m)))
    Xm = np.absolute(X); Xp = np.angle(X)
    # Espectro de magnitud:
    ax[1].cla(); ax[1].plot(f, Xm); 
    # Espectro de fase enmascarado con el espectro de magnitud
    ax[2].cla(); ax[2].plot(f, Xm*Xp/np.amax(Xm)); ax[2].set_ylim([-np.pi, np.pi])
    angle_delay = Xp[np.argmax(Xm)]
    ax[2].set_title("%0.4f [rad], %0.0f [deg]" % (angle_delay, 180*angle_delay/np.pi))

#interact(update, m=IntSlider_nice(min=0, max=20));
anim = animation.FuncAnimation(fig, update, frames=40, interval=100, blit=True)
#plt.close(); HTML(anim.to_html5_video())

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(6, 3))
Fs=22050; n = np.arange(0, 4, step=1.0/Fs) 
x = lambda m: np.sin(2.0*np.pi*880*(n-m))*np.exp(-(n-m)**2/0.5**2)*np.heaviside(n-m, 0)
y = x(0)  #+ 0.5*x(1.)   + 0.25*x(2.) + 0.125*x(3.)
ax.plot(n, y);
Audio(clean_4_audio(y), rate=Fs)

- El eco en cambio si modifica el espectro de magnitud
- Notemos el efecto de interferencia constructiva y destructiva al modificar el retardo
- La señal retardada cancela la original cuando tiene la misma amplitud ($A=1$)

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(6, 6))
n = np.arange(0, 400, step=1)
x = lambda m: np.sin(2.0*np.pi*0.05*(n-m)) 
f = fftpack.fftshift(fftpack.fftfreq(d=1, n=len(n)))
A = 1.
def update(m):
    m = 0.5*m; y = x(0) + A*x(m)
    ax[0].cla(); ax[0].plot(n, x(0), n, A*x(m))
    ax[1].cla(); ax[1].plot(n, y); ax[1].set_ylim([-A-1.1, A+1.1])
    X = fftpack.fftshift(fftpack.fft(y))
    ax[2].cla(); ax[2].plot(f, np.absolute(X)); 
    ax[2].set_ylim([-20, (1+A)*len(n)/2 + 20])
#interact(update, m=IntSlider_nice(min=0, max=20), 
#         A=SelectionSlider_nice(options=[0.5, 1., 2.], value=1));
anim = animation.FuncAnimation(fig, update, frames=40, interval=100, blit=False)

Notemos que en el caso $A=1$

$$
\begin{align}
y[n] &= \sin(2\pi k_0 n) + \sin(2\pi k_0 (n-m)) \nonumber \\ 
&= 2 \cos(\pi k_0 m) \sin(2\pi k_0 (n-m/2))  \nonumber
\end{align}
$$

se anula si

$$
\begin{align}
\pi k_0 m & = \frac{\pi}{2} + \pi i, \quad i \in \mathbb{Z} \nonumber \\
m &= \frac{1 + 2i}{2 k_0}, \quad i \in \mathbb{Z} \nonumber
\end{align}
$$

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(6, 3))
n = np.arange(0, 4, step=1.0/22050)
x = lambda m: np.sin(2.0*np.pi*880*(n-m))*np.exp(-(n-m)**2/0.5**2)*np.heaviside(n-m, 0)
y = x(0) + 1.*x(1./(2*880))
ax.plot(n, y);
Audio(clean_4_audio(y), rate=22050)

https://www.youtube.com/watch?v=IU8xeJlJ0mk

[&larr; Volver al índice](#index)

***
<a id="section2"></a>

# Sistema FIR 

***

Generalizando el ejemplo de sistema lineal reverberante a más retardos llegamos a 

$$
\begin{align}
y[n] &= h[0] x[n] + h[1] x[n-1] + h[2] x[n-2] + \ldots + h[L] x[n-L] \nonumber \\
&= \sum_{j=0}^{L} h[j] x[n-j] \nonumber \\
&= (h* x)[n] \nonumber 
\end{align}
$$

que se puede modelar como una convolución discreta o en pseuco-código como un ciclo iterativo
```
   y[n] = 0
   for j in 0 to L
       y[n] = y[n] + h[j] x[n-j]
```
y se conoce como


- sistema FIR (finite impulse response)
- sistema MA (moving average)
- sistema todo-zeros 


y es de orden L (posee L+1 coeficientes)

- ¿Es este sistema lineal?
- ¿Que ocurre si entra al sistema un impulso unitario?

## Intepretación como media movil (MA)

- El sistema FIR es equivalente a una media movil ponderada (*weighted moving average*)
- Los coeficientes del sistema son los ponderadores 

Por ejemplo sea un sistema de 3 coeficientes unitarios
$$
\begin{align}
y[n] = (h*x)[n] &= \sum_{j=0}^{2} h[j] x[n-j] \nonumber \\
&= x[n] + x[n-1] + x[n-2] \nonumber
\end{align}
$$
donde cada salida se calcula a partir de 
$$
\overbrace{x[0], x[1], x[2]}^{y[2]} , x[3], x[4], \ldots
$$
$$
x[0], \overbrace{x[1], x[2] , x[3]}^{y[3]}, x[4], \ldots
$$
$$
x[0], x[1], \overbrace{x[2] , x[3], x[4]}^{y[4]}, \ldots
$$

En este caso para obtener el valor de $y[0]$ e $y[1]$ se deben establecer "condiciones de borde"

### Ejemplo: Eliminando ruido blanco aditivo

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(9, 7))
np.random.seed(0); n = np.arange(0, 100, step=1)
C = 5*np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/10**2)
x_clean = np.random.multivariate_normal(np.zeros_like(n), C) 
ax[0].plot(n, x_clean)
x = x_clean + 2*np.random.randn(len(n))
ax[0].plot(n, x, 'k.'); ylims = ax[0].get_ylim()
# System:
L = 10; h = np.ones(shape=(L,))/L; 
# Acumulator
y = np.zeros_like(n, dtype=np.float)

def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    
def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+L] = h
    ax[1].cla(); ax[1].plot(n, c); 
    y[m] = np.sum(h*x[m:m+L])
    ax[2].cla(); ax[2].plot(n, y);  ax[2].set_ylim(ylims)
    ax[2].plot([m, m], [ylims[0], ylims[1]], 'r--', alpha=0.5)

anim = animation.FuncAnimation(fig, update,init_func=init, 
                               frames=100-len(h), interval=200, blit=True)

En la práctica podemos usar [`scipy.signal.convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html)

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(8, 6))
ax[0].plot(n, x_clean); ax[0].plot(n, x, 'k.')
L = 10; 
h = np.ones(shape=(L,)); 
#h = scipy.signal.tukey(L)
#h = scipy.signal.hamming(L)
h = h/np.sum(h)
ax[1].plot(h)
ax[2].plot(scipy.signal.convolve(x, h, mode='same', method='auto')); 
ax[2].set_ylim(ylims);

### Ejemplo: Encontrando cambios en una señal

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(8, 6))
n = np.arange(0, 100, step=1)
x = np.zeros_like(n, dtype=np.float)
x[20:] += 1.; x[40:] += 1.; x[80:] -= 1.;
ax[0].plot(n, x)
# System:
h = np.array([-0.5, 0.5])
# Acumulator
y = np.zeros_like(n, dtype=np.float)
def init():
    global y
    y = np.zeros_like(n, dtype=np.float)

def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+len(h)] = h
    ax[1].cla(); ax[1].plot(n, c); 
    y[m] = np.sum(h*x[m:m+len(h)])
    ax[2].cla(); ax[2].plot(n, y);  
    ax[2].plot([m, m], [-0.5, 0.5], 'r--', alpha=0.5)

anim = animation.FuncAnimation(fig, update, init_func=init, frames=100-len(h), interval=200, blit=True)

### Ejemplo: Removiendo una tendencia

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(8, 6))
np.random.seed(0); n = np.arange(0, 100, step=1)
C = np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/30**2)
x = np.sin(2.0*np.pi*0.1*n) + 5*np.random.multivariate_normal(np.zeros_like(n), C)
ax[0].plot(n, x); ylims = ax[0].get_ylim()
# System:
L = 5; h = -np.ones(shape=(L,))/L; h[L//2] += 1
# Acumulator
y = np.zeros_like(n, dtype=np.float)
def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    
def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+len(h)] = h
    ax[1].cla(); ax[1].plot(n, c); 
    y[m] = np.sum(h*x[m:m+len(h)])
    ax[2].cla(); ax[2].plot(n, y);  
    ax[2].plot([m, m], [-0.5, 0.5], 'r--', alpha=0.5)
    
anim = animation.FuncAnimation(fig, update, init_func=init, 
                               frames=100-len(h), interval=200, blit=True)

### Ejemplo: Detectando una patrón específico en una señal ruidosa

In [ ]:
def sine_pulse(n, k0, center):
    x = np.zeros_like(n, dtype=np.float); L = int(1/k0)
    x[center:center+L] = np.sin(2.0*np.pi*k0*(n[center:center+L] - n[center]))
    return x

plt.close('all'); fig, ax = plt.subplots(3, figsize=(8, 6))
np.random.seed(0); n = np.arange(0, 100, step=1)
x_clean = sine_pulse(n, k0=0.05, center=50) 
ax[0].plot(n, x_clean)
s = 1.;  x = x_clean + s*np.random.randn(len(n))
ax[0].plot(n, x, 'k.'); ylims = ax[0].get_ylim()
ax[0].set_title("SNR %0.2f [dB]" %(10*np.log(0.5/s**2)))
# System:
h = np.sin(2.0*np.pi*0.05*n[:int(len(n)/5)] + np.pi); h = h/np.sum(np.absolute(h))
# Acumulator
y = np.zeros_like(n, dtype=np.float)
def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    
def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+len(h)] = h
    ax[1].cla(); ax[1].plot(c); 
    y[m] = np.sum(h*x[m:m+len(h)])
    ax[2].cla(); ax[2].plot(n, y**2);  #ax[2].set_ylim(ylims)
    ax[2].plot([m, m], [np.amin(y**2), np.amax(y**2)], 'r--', alpha=0.5)

anim = animation.FuncAnimation(fig, update, init_func=init, 
                               frames=100-len(h), interval=200, blit=True)

***

# Sistema invariante al desplazamiento

***

Un sistema es invariante al desplazamiento  (*shift-invariant*) de su entrada si 

$$
f(x[n-m]) = y[n-m] 
$$

Es decir que 
- Aplicar un retardo en la entrada provoca un retardo equivalente en la salida
    - Un *blip* en la entrada produce un *blop* en la salida sin importar **cuando** ocurre el blip
- Las características del sistema no cambian con $n$
- Si $n$ representa el tiempo decimos que el sistema es **invariante en el tiempo**


***
Un sistema FIR es invariante al desplazamiento si los valores de $h$ son constantes para todo $n$

$$
h[n] = \text{Cte} \quad \forall n
$$

***

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 5))
n = np.arange(0, 500)
l0 = ax[0].plot(n, np.zeros_like(n)); 
ax[0].set_ylim([-0.1, 1.1]); ax[0].set_title('Input')
l1 = ax[1].plot(n, np.zeros_like(n));

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)      
    ax[1].set_ylim([-0.1, 1.1]); ax[1].set_title('Output')
    L = 50; h = np.ones(shape=(L, ))
    # h = 1.+ np.cos(2.0*np.pi*2.*(n-m)/len(n))
    h = h/np.sum(np.absolute(h))
    l0[0].set_ydata(x); l1[0].set_ydata(scipy.signal.convolve(x, h, mode='same'))

anim = animation.FuncAnimation(fig, update, frames=500, interval=200, blit=True);

***

# Respuesta al impulso de un sistema

***

Sea el impulso unitario o delta de Kronecker
$$
\delta[n-m] = \begin{cases} 1 & n=m \\ 0 & n \neq m \end{cases}
$$

La **respuesta al impulso de un sistema discreto** es la salida obtenida cuando la entrada es un impulso unitario

***

Para un sistema FIR arbitrario tenemos

$$
y[n]|_{x=\delta} = (h * \delta)[n] = \sum_{j=0}^L h[j] \delta[n-j] = \begin{cases} h[n] & n \in [0, L] \\ 0 & \text{en otro caso} \end{cases} \\
$$

es decir que la respuesta al impulso:
-  tiene **una duración finita y luego decae a zero**
-  recupera los coeficientes $h[j]$ del sistema

En un sistema causal se tiene que $h[n] = 0 \quad \forall n < 0$

Llamamos **soporte** del sistema a todos aquellos valores de $n$ tal que  $h[n] \neq 0$

***

**Ejemplo:** Para 
$$
y[n] = x[n] + A x[n-m]
$$
la respuesta al impulso es: 
$$
y[n] = \delta[n] + A \delta[n-m] = \begin{cases} 1 & n=0\\ A& n=m \\ 0 & \text{en otro caso} \end{cases}
$$

[&larr; Volver al índice](#index)

***
<a id="section3"></a>

# Sistema LTI 
***

Los sistemas LTI (*linear time-invariant*) cumplen 

- linealidad: homogeneo y aditivo
- invarianza al desplazamiento de la entrada

y se expresan como

$$
y[n] = (h * x)[n] = \sum_j h[j] x[n-j]
$$
- h es la respuesta al impulso
- h guarda los coeficientes del sistema

***
Por propiedad de la DFT sabemos que un sistema LTI cumple

$$
\begin{align}
\text{DFT}_N [y[n]] & = \text{DFT}_N [(h * x)[n]] \nonumber \\
\text{DFT}_N [y[n]] & = \text{DFT}_N [h[n]]  \text{DFT}_N [x[n]] \nonumber \\
Y[k] &= H[k] X[k] , \nonumber
\end{align}
$$

donde convertimos la convolución temporal en una multiplicación frecuencial


- Llamamos a $H[k]$ la **respuesta en frecuencia del sistema** 
- La respuesta en frecuencia es la transformada de Fourier de la respuesta al impulso

***

Los sistemas LTI tienen intepretación directa en el dominio del frecuencia

***

***

# Filtros digitales
***

Un **filtro** es un sistema cuyo objetivo es reducir o resaltar un cierto aspecto de una señal

Por ejemplo

- Disminuir el nivel de ruido
- Separar señales mezcladas
- Ecualizar
- Restaurar (eliminar desenfoque o artefactos de grabación)

Llamamos filtro digital a los filtros aplicados a señales digitales

Hablamos de "señal filtrada" para referirnos a la salida del filtro

A continuación estudiaremos filtros LTI, más adelante extenderemos a filtros adaptivos (no-TI) y filtros no-lineales (no-L)

***

El **filtro LTI** se puede estudiar en frecuencia usando 

$$
Y[k] = H[k] X[k] ,
$$

donde $H[k]$ es la DFT del filtro (respuesta en frecuencia)

- El **filtro LTI** actua como una máscara que modifica el espectro de la entrada
- Solo puede acentuar, atenuar o remover ciertas frecuencias pero **nunca crear nuevas frecuencias**

Consideremos los siguienes filtros (máscaras) ideales

<img src="img/ideal_filters.gif">

***

## Diseño de un filtro FIR 


Recordemos, un filtro FIR 

$$
\begin{align}
y[n] &= h[0] x[n] + h[1] x[n-1] + h[2] x[n-2] + \ldots + h[L] x[n-L] \nonumber \\
&= \sum_{j=0}^{L} h[j] x[n-j] \nonumber \\
&= (h* x)[n] \nonumber 
\end{align}
$$

está definido por su orden (L) y sus coeficientes $h[j], j=0,\ldots, L$
***
Diseñar un filtro consisten en encontrar L y los valores de los coeficientes
***
Podemos diseñar un filtro FIR siguiendo estos pasos

1. Especificar una respuesta en frecuencia ideal $H_d[k]$
1. Usar la transformada de Fourier inversa para obtener la respuesta al impulso ideal $h_d[n]$
1. Truncar la respuesta al impulso ideal usando una ventana tal que $h[n] = h_d[n] w[n]$
1. $h[n]$ nos da los coeficientes del filtro FIR y w[n] nos da el largo del filtro

La ventana $w[n]$ puede ser cualquiera de las funciones vistas en la unidad anterior, por ejemplo una ventana rectangular

$$
w[n] = \begin{cases} 1 & n \leq L \\ 0 & n > L \end{cases}
$$

o la ventana de Hann

$$
w[n] = 0.5 - 0.5 \cos \left( \frac{2\pi n}{L-1} \right)
$$



***

### Ejemplo: Filtro pasa bajo (LPF)

Un filtro pasa bajo es aquel que sólo deja pasar las **bajas** frecuencias

Sus usos son:
- Recuperar una tendencia o comportamiento lento en la señal
- Eliminar el ruido aditivo

Diseñemos un filtro que elimine todas las frecuencias mayores a $f_c$ [Hz] de una señal $x[n]$ muestreada con frecuencia $F_s$. 

1. Propongamos la siguiente respuesta en frecuencia ideal que solo deja pasar las frecuencias menores a $f_c$
$$
\begin{align}
H_d(\omega) &= \begin{cases} K & |f| < f_c\\ 0 & |f| > f_c \end{cases} \nonumber \\
&= K \text{rect}(f/f_c) \nonumber 
\end{align}
$$

1. Obtenemos su transformada de Fourier inversa
$$
\begin{align}
h_d(t) &= K \int_{-f_c}^{f_c} e^{j 2 \pi f t} df \nonumber \\
& = \frac{2j K f_c}{2 j \pi f_c t} \sin(2 \pi f_c t) = 2 K f_c \text{sinc}(2 \pi f_c t) \nonumber 
\end{align}
$$
que es una función infinitamente larga

1. La versión discreta sería
$$
h_d[n] = 2 K f_c\text{sinc}(2 \pi f_c n/ F_s) 
$$

1. Para obtener una respuesta al impulso finita multiplicamos por una ventana de largo finito

$$
h[n] = 2 K f_c \text{sinc}(2 \pi f_c n /F_s) \text{rect}(n/(L+1))
$$

que implica esto en la respuesta en frecuencia real?


In [ ]:
plt.close('all'); fig, ax = plt.subplots(5, 2, figsize=(8, 9))
n = np.arange(0, 100, step=1); 
f = fftpack.fftshift(fftpack.fftfreq(n=len(n), d=1))
np.random.seed(0); C = 5*np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/10**2)
x_clean = np.random.multivariate_normal(np.zeros_like(n), C)
x_clean -= np.mean(x_clean)
x = x_clean + 2*np.random.randn(len(n))
def update(fc, L, window='rect'):
    [axx.cla() for axx in ax.ravel()]
    # Señal limpia
    ax[0, 0].plot(n, x_clean); 
    ax[0, 1].plot(f, fftpack.fftshift(np.absolute(fftpack.fft(x_clean))))
    ax[0, 0].set_title("Tiempo"); ax[0, 1].set_title("Frecuencia")    
    # Señal contaminada
    ax[1, 0].plot(n, x); ylims = ax[1, 0].get_ylim(); ax[0, 0].set_ylim(ylims)
    X = fftpack.fft(x); 
    ax[1, 1].plot(f, fftpack.fftshift(np.absolute(X)))
    # Filtro ideal
    kc = int(len(n)*fc)
    Hd = np.zeros_like(n, dtype=np.float); Hd[:kc] = 1.; Hd[len(Hd)-kc+1:] = 1.
    ax[2, 1].plot(f, fftpack.fftshift(Hd))
    hd = np.real(fftpack.ifftshift(fftpack.ifft(Hd)))
    ax[2, 0].plot(n, hd)
    # Filtro real
    w = np.zeros_like(n, dtype=np.float); 
    if window == 'rect':
        w[len(w)//2-L//2:len(w)//2+L//2+1] = 1.;
    elif window == 'hann':
        w[len(w)//2-L//2:len(w)//2+L//2+1] = scipy.signal.hann(L+1)
    h = w*hd; H = fftpack.fft(h); 
    print("NMSE entre H y Hd %0.4f" % (np.mean((np.absolute(H) - Hd)**2)/np.mean(Hd**2)))
    ax[3, 0].plot(n, h);  ylims2 = ax[3, 0].get_ylim(); 
    ax[3, 0].plot(n, w, '--'); ax[3, 0].set_ylim(ylims2)
    ax[3, 1].plot(f, fftpack.fftshift(np.absolute(H)))
    # Señal filtrada
    xf = scipy.signal.convolve(x, h, mode='same')
    ax[4, 0].plot(n, np.real(xf)); 
    ax[4, 1].plot(f, fftpack.fftshift(np.absolute(fftpack.fft(xf)))); ax[4, 0].set_ylim(ylims)
    
interact(update, fc=FloatSlider_nice(min=0.01, max=0.5, step=0.01, value=0.1),
         L=SelectionSlider_nice(options=[4, 6, 10, 20, 40, 60, 80], value=60),
         window=SelectionSlider_nice(options=['rect', 'hann']));

- La respuesta en frecuencia "ideal" $H_d[k]$ es plana y tiene discontinuidades
- La respuesta en frecuencia "real" $H[k]$ busca aproximar a $H_d[k]$
- Observando $H[k]$ notamos que
    - No tiene zonas perfectamente planas (ni es perfectamente zero)
    - No tiene discontinuidades fuertes como el caso ideal
- Esto se debe al recorte que hacemos con la ventana

<img src="img/system-real-filter.png" width="500">

- Podemos usar una ventana suave (Hann) para disminuir los *ripples* al costo de hacer más lenta las transiciones
***
El diseño del filtro está dado entonces por su
- **Función:** Definida por la respuesta en frecuencia ideal $|H_d[k]| \angle H_d[k]$ 
    - Podemos definir la magnitud o el ángulo dependiendo de la aplicación
- **Fidelidad:** El error tolerable entre la respuesta en frecuencia ideal y la real
***

***

### Ejemplo: Filtro pasa alto (HPF)

Un filtro pasa alto es aquel que sólo deja pasar las **altas** frecuencias

Sus usos son:
- Identificar cambios/detalles, es decir comportamientos rápidos en una señal
- Eliminar tendencias

Diseñemos un filtro que elimine todas las frecuencias menores a $f_c$ [Hz] de una señal $x[n]$ muestreada con frecuencia $F_s$. 

1. Propongamos la siguiente respuesta en frecuencia ideal que solo deja pasar las frecuencias **mayores** a $f_c$
$$
H_d(\omega) = \begin{cases} K & |f| \geq f_c\\ 0 & |f| < f_c \end{cases} 
$$

1. Obtenemos su transformada de Fourier inversa
$$
\begin{align}
h_d(t) &= K \int_{-\infty}^{-f_c} e^{j 2 \pi f t} df  + K \int_{f_c}^{\infty} e^{j 2 \pi f t} df\nonumber \\
&= K \int_{-\infty}^{\infty} e^{j 2 \pi f t} df  - K \int_{-f_c}^{f_c} e^{j 2 \pi f t} df\nonumber \\
& = K \delta(t) - 2 K f_c \text{sinc}(2 \pi f_c t) \nonumber 
\end{align}
$$

1. La versión discreta sería
$$
h_d[n] = K (\delta[n] - 2 F_c\text{sinc}(2 \pi f_c n/ F_s) )
$$

1. Finalmente la hacemos finita multiplicando por una ventana de largo finito
$$
h[n] = K (\delta[n] - 2 f_c\text{sinc}(2 \pi f_c n/ F_s) ) \text{rect}(n/(L+1))
$$

***
- Un filtro pasa alto se obtiene de forma trivial **restandole un filtro pasa bajo a un impulso unitario**
- Este "truco" se conoce como  **inversión espectral**


<img src="img/system-hpf.png" width="400">
***


***

### Ejemplo: Filtro pasa banda (BPF) y rechaza banda (BRF)

- Como sus nombres lo indicam estos filtros 
    - Dejan pasar una cierta banda de frecuencia (BPF) o
    - Dejan pasar todas las frecuencias excepto una banda determinada (BRF)
- La banda de frecuencia está definida por sus frecuencias de corte mínima y máxima 
- Se pueden construir combinando el filtro pasa bajo y pasa alto

**Filtro pasa banda**: 
Para recuperar solo una cierta banda de frecuencia seguimos:
1. Definir frecuencias de corte $f_{c1} < f_{c2}$
- Construir un filtro pasa bajo con frecuencia de corte $f_{c2}$
- Convolucionar la señal con la respuesta al impulso resultante
- Construir un filtro pasa alto con frecuencia de corte $f_{c1}$
- Convolucionar la señal filtrada con la respuesta al impulso resultante

<img src="img/system-bpf.png" width="400">

**Filtro rechazabanda**: 
Para eliminar una cierta banda de frecuencia seguimos:
1. Definir frecuencias de corte $f_{c1} > f_{c2}$
- Construir un filtro pasa bajo con frecuencia de corte $f_{c2}$
- Construir un filtro pasa alto con frecuencia de corte $f_{c1}$
- Sumar ambos las respuestas al impulso de ambos filtros
- Convolucionar la señal con la respuesta al impulso resultante

<img src="img/system-rbf.png" width="400">

In [ ]:
plt.close('all'); fig, ax = plt.subplots(4, figsize=(8, 8), tight_layout=True)

def low_pass_h(L, fc, fs):
    h = np.sinc(2*fc*np.arange(-L//2, L//2)/fs) 
    return h/np.sum(h) # ganancia unitaria K = 1/(2fc sum h)

L = 200+1; fc_lp, fc_hp, fs = 2, 3, 10
h_lp = low_pass_h(L, fc_lp, fs)
h_hp = -low_pass_h(L, fc_hp, fs); h_hp[L//2+1] += 1

h = h_lp
# h = h_hp + h_lp
# h = scipy.signal.convolve(h_hp, h_lp)

ax[0].plot(h); ax[0].set_ylabel("h[n]")
freq, response = scipy.signal.freqz(h)
ax[1].plot(0.5*fs*freq/np.pi, np.abs(response));
ax[2].plot(0.5*fs*freq/np.pi, np.unwrap(np.angle(response)));
freq, delay = scipy.signal.group_delay((h, 1))
ax[3].plot(0.5*fs*freq/np.pi, delay); ax[3].set_ylabel("Retardo")
ax[3].set_ylim([-50, 150])
ax[1].set_ylabel("|H[k]|"); ax[2].set_ylabel("angle H[k]"); 

In [ ]:
data, sample_rate = sf.read("data/nomekop.ogg")
plt.close('all'); fig, ax = plt.subplots(1, figsize=(8, 3))
freq, ttime, Sxx = scipy.signal.spectrogram(data, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax.set_ylim([0.0, 1e+4]);
Audio(data, rate=int(sample_rate))

`scipy.signal.firwin(numtaps, cutoff, width=None, window='hamming', pass_zero=True, scale=True, nyq=None, fs=None)`
- numtaps (entero): Largo del filtro 
- cutoff (arreglo flotante): frecuencias de corte
- window (string): tipo de ventana
- pass_zero (bool): pasa bajo o pasa alto
- fs (flotante): Frecuencia de muestreo

In [ ]:
h = scipy.signal.firwin(numtaps=1000+1, cutoff=[200, 500], 
                        pass_zero=False, window='hann',
                        fs=sample_rate)

plt.close('all'); fig = plt.figure(figsize=(8, 6))
ax1 = plt.subplot2grid((2, 2 ), (0, 0)); ax1.set_title("Respuesta al impulso")
ax2 = plt.subplot2grid((2, 2 ), (0, 1)); ax2.set_title("Respuesta en frecuencia")
ax3 = plt.subplot2grid((2, 2 ), (1, 0), colspan=2, rowspan=1)

freq, response = scipy.signal.freqz(h)
ax1.plot(h); ax2.semilogy(0.5*sample_rate*freq/np.pi, np.abs(response));
data_filt = scipy.signal.convolve(data, h)
freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax3.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax3.set_ylim([0.0, 1e+4]);
Audio(data_filt, rate=int(sample_rate))

***

### Filtro FIR con respuesta en frecuencia arbitraria

Consideremos un filtro FIR 
- De orden $L$ (par) con $L+1$ coeficientes 
- Centrado en el origen (cero fase)
- Con respuesta al impulso simétrica: $h_{n} = h_{-n}$

Podemos escribir su respuesta en frecuencia como 

$$
H[k] = \sum_{n=-L/2}^{L/2} h[n] e^{-j \frac{2\pi}{N} k n} = h[0] + \sum_{n=1}^{L/2} h[n]\cos \left(\frac{2\pi}{N} k n\right)
$$
que para $k=0, \ldots, N-1$ en forma matricial es

$$ 
\begin{align} 
\begin{pmatrix} H[0] \\ H[1] \\ H[2] \\ \vdots \\ H[N-1] \\ \end{pmatrix} 
&= 
\begin{pmatrix} 
1 & 1 & 1 & \cdots & 1 \\ 
1 & \cos \left(\frac{2\pi}{N} \right) & \cos \left(\frac{2\pi}{N} 2\right) & \cdots & \cos \left(\frac{2\pi}{N} L/2\right) \\ 
1 & \cos \left(\frac{2\pi}{N} 2 \right) & \cos \left(\frac{2\pi}{N} 4 \right) & \cdots & \cos \left(\frac{2\pi}{N} 2 L/2\right) \\ 
\vdots & \dots & \dots & \ddots & \vdots \\ 
1 & \cos \left(\frac{2\pi}{N} (N-1)\right) & \cos \left(\frac{2\pi}{N} (N-1) 2\right) & \cdots & \cos \left(\frac{2\pi}{N} (N-1) L/2\right) \\ 
\end{pmatrix} 
\begin{pmatrix} h[0] \\ h[1] \\ h[2] \\ \vdots \\ h[L/2] \\ \end{pmatrix} \nonumber \\
H &= A h, \nonumber
\end{align} 
$$

donde $A \in \mathbb{R}^{NxL/2}$

Si especificamos $H$ podemos despejar $h$ pero no podemos invertir A ya que no es cuadrada.
***
Una alternativa es buscar $\hat h \approx h$ que minimice el error cuadrático medio

$$
\min_h \| H - Ah\|^2,
$$
derivando e igualando a cero obtenemos el sistema de **ecuaciones normales**

$$
\begin{align}
\frac{d}{dh} \| H - Ah\|^2 &= 0 \nonumber \\
- 2 A^T (H - Ah) &= 0 \nonumber \\
A^T A h &= A^T H \nonumber \\
\hat h &= (A^T A)^{-1} A^T H  \nonumber \\
\hat h &= A^{\dagger} H, \nonumber
\end{align}
$$

donde 
- $A^{\dagger} = (A^T A)^{-1} A^T$ se conoce como la pseudo-inversa de A
- $\hat h$ se conoce como el **estimador de mínimos cuadrados** de h

***

`scipy.signal.firls(numtaps, bands, desired, weight=None, nyq=None, fs=None)`

- numtaps (entero impar): número de coeficientes
- bands: secuencia creciente de frecuencias
- desired: secuencia creciente de ganancias
- fs (flotante): Frecuencia de muestreo

In [ ]:
# Estimador de minimos cuadrados para filtro FIR usando scipy.signal
h = scipy.signal.firls(numtaps=1000+1, 
                       bands=(0., 640, 650, 2340, 2350, 2750, 2760, sample_rate//2), 
                       desired=(5., 5., 0., 0., 1., 1., 0., 0.), fs=sample_rate)

plt.close('all'); fig = plt.figure(figsize=(8, 6))
ax1 = plt.subplot2grid((2, 2 ), (0, 0)); ax1.set_title("Respuesta al impulso")
ax2 = plt.subplot2grid((2, 2 ), (0, 1)); ax2.set_title("Respuesta en frecuencia")
ax3 = plt.subplot2grid((2, 2 ), (1, 0), colspan=2, rowspan=1)

freq, response = scipy.signal.freqz(h)
ax1.plot(h); ax2.semilogy(0.5*sample_rate*freq/np.pi, np.abs(response));
data_filt = scipy.signal.convolve(data, h)
freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax3.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax3.set_ylim([0.0, 1e+4]);
Audio(data_filt, rate=int(sample_rate))

[&larr; Volver al índice](#index)

Un filtro FIR de buena calidad puede requerir una gran cantidad de coeficientes

Es posible implementar filtros más eficientes usando recursividad

***
<a id="section4"></a>

# Sistema IIR (infinite impulse response)

***

Generalizando el sistema FIR para incluir versiones pasadas de la salida y asumiendo $a[0] = 1$ llegamos a 

$$
\begin{align}
y[n] &= b[0] x[n] + b[1] x[n-1] + b[2] x[n-2] + \ldots + b[L] x[n-L]  \nonumber \\
& - a[1] y[n-1] - a[2] y[n-2] - \ldots - a[M] y[n-M] \nonumber \\
&= \sum_{l=0}^{L} b[l] x[n-l] - \sum_{m=1}^{M} a[m] y[n-m]  \nonumber  \\
\sum_{m=0}^{M} a[m] y[n-m] &= \sum_{l=0}^{L} b[l] x[n-l] \nonumber \\
(a * y)[n] &= (b * x)[n], \nonumber
\end{align}
$$

es decir dos convoluciones discretas que definen una **ecuación de diferencias**

Este tipo de sistema se conoce como 
- sistema IIR (infinite impulse response)
- sistema *auto-regresive moving average* (ARMA)
    - autoregresivo de orden M: incluye valores pasados de la salida
    - media movil de orden L+1: pondera el valor presente y pasados de la entrada

¿Cuando se recupera un sistema FIR?

R: Si $a[m] = 0$ para $m=[1, \ldots, M]$

***

## Respuesta en frecuencia del sistema IIR

Aplicando la transformada de Fourier convertimos las convoluciones en multiplicaciones y encontramos la respuesta en frecuencia como

$$
\begin{align}
\text{DFT}_N[(a * y)[n]] &= \text{DFT}_N[(b * x)[n]] \nonumber \\
A[k] Y[k] &= B[k] X[k] \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{B[k]}{A[k]} = \frac{ \sum_{l=0}^L b[l]e^{-j \frac{2\pi}{N} nl} }{ \sum_{m=0}^M a[m]e^{-j \frac{2\pi}{N} mk}} \nonumber
\end{align}
$$
siempre que $A[k] \neq 0$. 

Se llega a un resultado equivalente usando la propiedad de traslación de la DFT

$$
\begin{align}
\text{DFT}_N\left[\sum_{m=0}^{M} a[m] y[n-m]\right] &= \text{DFT}_N\left[\sum_{l=0}^{L} b[l] x[n-l]\right] \nonumber \\
\sum_{m=0}^{M} a[m] \mathbb{DFT}_N\left[y[n-m]\right] &= \sum_{l=0}^{L} b[l] \mathbb{DFT}_N\left[x[n-l]\right] \nonumber \\
\sum_{m=0}^{M} a[m] Y[k] e^{-j \frac{2\pi}{N} km} &= \sum_{l=0}^{L} b[l] X[k] e^{-j \frac{2\pi}{N} kl} \nonumber \\
Y[k] \sum_{m=0}^{M} a[m] e^{-j \frac{2\pi}{N} km} &= X[k] \sum_{l=0}^{L} b[l] e^{-j \frac{2\pi}{N} kl} \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{ \sum_{l=0}^L b[l]e^{-j \frac{2\pi}{N} lk} }{ \sum_{m=0}^M a[m]e^{-j \frac{2\pi}{N} mk}} \nonumber
\end{align}
$$

que también puede expresarse de forma alternativa como 

$$
H[k] = K \frac{ \prod_{l=1}^L (e^{j \frac{2\pi}{N} k}- \beta[l]) }{ \prod_{m=1}^M (e^{j \frac{2\pi}{N} k}- \alpha[m])} 
$$

donde $K$ es la **ganancia** y las raices de los polinomios se llaman **ceros** (numerador) y **polos** (denominador)

### Ejemplo:

Consideremos el siguiente sistema IIR 

$$
\begin{align}
y[n] &= (1-\gamma) x[n] + \gamma y[n-1] \nonumber \\
y[n] - \gamma y[n-1] &= (1-\gamma) x[n] \nonumber
\end{align}
$$

¿Cuántos coeficientes tiene este sistema?

R: $a[0] = 1$, $a[1] = -\gamma$ y $b[0] = (1-\gamma)$, es decir MA de orden 1 y AR de orden 1

¿Cúal es su respuesta al impulso? (asumiendo $y[n]=0, n<0$)

$$
\begin{matrix}
n & \delta[n] & y[n] \\
-2 & 0 & 0 \\
-1 & 0 & 0 \\
0 & 1 & (1-\gamma) \\
1 & 0 & \gamma(1-\gamma) \\
2 & 0 & \gamma^2(1-\gamma)  \\
3 & 0 & \gamma^3(1-\gamma) \\
4 & 0 & \gamma^4(1-\gamma) \\
\end{matrix}
$$

¿Qué pasa si $\gamma \geq 1$?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, figsize=(9, 5))

def update(gamma):
    t, y = scipy.signal.dimpulse(([1-gamma, 0], [1, -gamma], 1), x0=0, n=30)
    ax.cla(); ax.plot(t, y[0], lw=4); 

interact(update, gamma=FloatSlider_nice(min=-1.1, max=2.1, step=0.01), msg_throttle=1);

- La respuesta al impulso puede tender a infinito!
- A diferencia de un sistema FIR el sistema IIR puede tener una configuración
    - Estable: $h$ tiende a cero
    - Condicionalmente estable: $h$ oscila a una taza constante
    - Inestable: $h$ crece/decrece infinitamente

Consideremos el sistema anterior y asumamos que $|\gamma|<1$, desenrollando tenemos que 

$$
\begin{align}
y[0] &= (1-\gamma) x[0] \nonumber \\
y[1] &= (1-\gamma) (x[1] + \gamma x[0]) \nonumber \\
y[2] &= (1-\gamma) (x[2] + \gamma x[1] + \gamma^2 x[0]) \nonumber \\
y[3] &= (1-\gamma) (x[3] + \gamma x[2] + \gamma^2 x[1] + \gamma^3 x[0]) \nonumber \\
y[4] &= (1-\gamma) (x[4] + \gamma x[3] + \gamma^2 x[2] + \gamma^3 x[1]  + \gamma^4 x[0]) \nonumber \\
y[5] &= \ldots \nonumber 
\end{align}
$$

Con un sistema IIR de 3 coeficientes podemos representar un sistema FIR considerablemente más grande

Por ejemplo si escogemos $\gamma$ tal que $\gamma^{20 }\approx 0$ entonces aproximamos un sistema FIR de orden 20

Por otra parte la respuesta en frecuencia de este sistema

$$
\begin{align}
Y[k] &= (1-\gamma) X[k] + \gamma Y[k] e^{-j \frac{2\pi}{N} k} \nonumber \\
H[k] = \frac{Y[k]}{X[k]} &= \frac{1-\gamma}{1 - \gamma e^{-j \frac{2\pi}{N} k} } = (1-\gamma)\frac{e^{j \frac{2\pi}{N} k} - 0}{e^{j \frac{2\pi}{N} k} - \gamma  } \nonumber 
\end{align}
$$

que tiene un cero en $0$, un polo en $\gamma$ y ganancia $1-\gamma$

Para entender mejor este sistema estudiemos la magnitud de $|H[k]|$ para $\gamma < 1$

$$
\begin{align}
| H[k]| &= \frac{|1-\gamma|}{|1 - \gamma e^{-j \frac{2\pi}{N} k}|}  \nonumber \\
&=  \frac{1-\gamma}{\sqrt{1 - 2\gamma \cos(\frac{2\pi}{N} k) + \gamma^2}} \nonumber
\end{align}
$$

¿Cómo se ve $|H[k]|$? ¿Qué función cumple este sistema?

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, figsize=(9, 5))
k = np.arange(-24, 25)/50
Hk = lambda gamma, k : (1-gamma)/np.sqrt(1 - 2*gamma*np.cos(2.0*np.pi*k) + gamma**2)
l = ax.plot(k, np.zeros_like(k), lw=4); ax.set_ylim([-0.05, 1.05]); ax.set_xlabel('Frequency')
def update(gamma):
    l[0].set_ydata(Hk(gamma, k));
    ax.set_title(r"$\gamma$: %0.2f" %(gamma))
interact(update, gamma=FloatSlider_nice(min=0., max=0.99, step=0.01), msg_throttle=1);

- Este sistema actua como filtro pasa bajo!

## Diseño de filtros IIR simples

Los filtros IIR más simples son los de un polo y un cero

$$
H[k] = \frac{b[0] + b[1] e^{-j \frac{2\pi}{N} k}}{1 + a[1] e^{-j \frac{2\pi}{N} k}}  = K\frac{e^{j \frac{2\pi}{N} k} - \beta}{e^{j \frac{2\pi}{N} k} - \alpha  } 
$$

donde podemos reconocer $b[0]=K$, $\beta = - b[1]K$ y $\alpha=-a[1]$

- Definimos la frecuencia de corte $f_c$ como aquella frecuencia en la que el filtro alcanza una atenuación de 0.7 (-3 dB)
- En este caso $\gamma = e^{-2\pi f_c}$

**Receta para un filtro pasa bajo IIR con frecuencia de corte $f_c$**

- $b[0] = 1 - e^{-2\pi f_c}$
- $b[1] = 0$
- $a[1] = -e^{-2\pi f_c}$

$$
H[k] = \frac{1-e^{-2\pi f_c}}{1 - e^{-2\pi f_c} e^{-j \frac{2\pi}{N} k}} = (1-e^{-2\pi f_c}) \frac{(e^{j \frac{2\pi}{N} k}- 0)}{(e^{j \frac{2\pi}{N} k} - e^{-2\pi f_c} )}
$$
- Un cero en $0$, un polo en $e^{-2\pi f_c}$ y ganancia $1-e^{-2\pi f_c}$

**Receta para un filtro pasa alto IIR con frecuencia de corte $f_c$**
- $b[0] = (1 + e^{-2\pi f_c})/2$
- $b[1] = -(1 + e^{-2\pi f_c})/2$
- $a[1] = -e^{-2\pi f_c}$

$$
H[k] = \frac{1+e^{-2\pi f_c}}{2} \frac{(e^{j \frac{2\pi}{N} k} - 1)}{(e^{j \frac{2\pi}{N} k} - e^{-2\pi f_c})}
$$
- Un cero en $1$, un polo en $e^{-2\pi f_c}$ y ganancia $\frac{1+e^{-2\pi f_c}}{2}$



Respuesta en frecuencia de un filtro:

`scipy.signal.freqz(b, a=1, worN=512, whole=False, plot=None)`
- b (arreglo): coeficientes del numerador de $H[k]$
- a (arreglo): coeficientes del denominador de $H[k]$

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(9, 7))

def update(fc):
    gamma = np.exp(2*np.pi*fc)
    b, a = [(1-gamma), 0], [1, -gamma]
    #gamma = np.exp(-2*np.pi*(fc))
    #b, a = [(1+gamma)/2, -(1+gamma)/2], [1, -gamma]
    freq, response = scipy.signal.freqz(b, a)
    ax[0].cla(); ax[0].plot(0.5*freq/np.pi, np.abs(response))
    ax[0].set_ylabel('Espectro de\n magnitud')
    ax[1].cla(); ax[1].plot(0.5*freq/np.pi, np.unwrap(np.angle(response)));
    ax[1].set_ylabel('Espectro de\n ángulo')
    freq, delay = scipy.signal.group_delay(system=(b, a))
    ax[2].cla(); ax[2].plot(0.5*freq/np.pi, delay); ax[2].set_ylabel("Retardo")

interact(update, fc=FloatSlider_nice(min=0.01, max=0.49, step=0.01, value=0.01));

- Su retardo no es igual para todas las frecuencias! ¿Qué consecuencia tiene esto?


Aplicar un filtro a una señal:

`scipy.signal.lfilter(b, a, x, axis=-1, zi=None)`
- b (arreglo): coeficientes del numerador
- a (arreglo): coeficientes del denominador
- x (arreglo): señal de entrada

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 7))
fc = 0.05; gamma = np.exp(-2*np.pi*(fc))
# b, a = [(1-gamma), 0], [1, -gamma]  # Low Pass
b, a = [(1+gamma)/2, -(1+gamma)/2], [1, -gamma]  # High pass
n = np.arange(0, 500)
l0 = ax[0].plot(n, np.zeros_like(n)); ax[0].set_ylim([-0.05, 1.05]); ax[0].set_title('Input')
l1 = ax[1].plot(n, np.zeros_like(n)); ax[1].set_ylim([-0.5, 1.05]); ax[1].set_title('Output')

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)
    l0[0].set_ydata(x);
    y = scipy.signal.lfilter(b, a, x)
    l1[0].set_ydata(y);

anim = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=True); 
#plt.close(); HTML(anim.to_html5_video())

# Diseño de filtros IIR de segundo orden

Los filtros IIR de segundo orden o **biquad** tienen dos polos y dos ceros.

Su respuesta en frecuencia es
$$
H[k] = \frac{b[0] + b[1] W_N^k + b[2] W_N^{2k}}{1 + a[1] W_N^k + a[2] W_N^{2k}} = K \frac{(W_N^{-k} - \beta_1) (W_N^{-k} - \beta_2)}{(W_N^{-k} - \alpha_1)(W_N^{-k} - \alpha_2)},
$$
donde $W_N = e^{-j \frac{2 \pi}{N}}$ y la relación entreo coeficientes y polos/ceros es: 
$$
b[0] = K, \quad b[1] = -K (\beta_1 + \beta_2), \quad b[2]= K \beta_1\beta_2
$$
$$
a[1] = - (\alpha_1 + \alpha_2), \quad a[2]=\alpha_1 \alpha_2
$$


- Con arquitecturas de segundo orden se pueden crear filtros pasabanda y rechaza banda
- Los filtros IIR de primer y segundo orden son menos propensos a inestabilidades
- https://www.dsprelated.com/showarticle/1137.php


In [ ]:
#https://www.slideshare.net/op205/basics-of-digital-filters-presentation

plt.close('all'); fig, ax = plt.subplots(3, figsize=(8, 6))

def update(fc1, fc2):
    gamma1 = np.exp(-2*np.pi*fc1)
    # gamma2 = np.exp(-2*np.pi*fc2)
    # b, a = [(1-gamma), 0], [1, -gamma]
    #b, a = [(1+gamma)/2, -(1+gamma)/2], [1, -gamma]
    z = [1, -1]
    p = [0.9*np.exp(-1j*2*np.pi*fc1), 0.9*np.exp(1j*2*np.pi*fc1)]
    k=1
    freq, response = scipy.signal.freqz_zpk(z=z, k=k, p=p)
    ax[0].cla(); ax[0].plot(0.5*freq/np.pi, np.abs(response))
    ax[0].set_ylabel('Espectro de\n magnitud')
    ax[1].cla(); ax[1].plot(0.5*freq/np.pi, np.unwrap(np.angle(response)));
    ax[1].set_ylabel('Espectro de\n ángulo')
    #freq, delay = scipy.signal.group_delay(system=(b, a))
    #ax[2].cla(); ax[2].plot(0.5*freq/np.pi, delay); ax[2].set_ylabel("Retardo")

interact(update, fc1=FloatSlider_nice(min=0.01, max=0.49, step=0.01, value=0.25),
         fc2=FloatSlider_nice(min=0.01, max=0.49, step=0.01, value=0.3));

# Diseño de filtros IIR de orden mayor

Crear coeficientes de filtro IIR de orden mayor

`scipy.signal.iirfilter(N, Wn, rp=None, rs=None, btype='bandpass', analog=False, ftype='butter', output='ba')`

- N (entero): orden del filtro
- Wn (arreglo): Frecuencias de corte (normalizadas en [0, 1])
- btype (string): Tipo de filtro {'bandpass', 'lowpass', 'highpass', 'bandstop'}
- ftype (string): Tipo de filtro {'butter', 'ellip', 'cheby1', 'cheby2', 'bessel'}
- output (string): Tipo de retorno

El filtro Butterworth ('butter') es óptimo en el sentido de tener la banda de paso lo más plana posible. Otros filtros se diseñaron con otras consideraciones. Los filtros IIR digitales están basados en los filtros IIR analógicos.

In [ ]:
plt.close('all'); fig, ax = plt.subplots(3, figsize=(9, 7))

def update(order, ftype):
    b, a = scipy.signal.iirfilter(N=order, Wn=0.2, ftype=ftype, btype='lowpass', output='ba')
    freq, response = scipy.signal.freqz(b, a)
    ax[0].cla(); ax[0].plot(freq/np.pi, np.abs(response))
    ax[0].set_ylabel('Espectro\nde magnitud')
    ax[1].cla(); ax[1].plot(freq/np.pi, np.unwrap(np.angle(response)));
    ax[1].set_ylabel('Espectro\nde ángulo')
    freq, delay = scipy.signal.group_delay(system=(b, a))
    ax[2].cla(); ax[2].plot(freq/np.pi, delay); ax[2].set_ylabel("Retardo")
    
interact(update, order=SelectionSlider_nice(options=[1, 2, 3, 5, 10, 15, 25], value=1),
         ftype=SelectionSlider_nice(options=['butter', 'bessel']));

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(8, 6))

b, a = scipy.signal.iirfilter(N=10, Wn=0.03, ftype='butter', btype='lowpass', output='ba')
n = np.arange(0, 500)
l0 = ax[0].plot(n, np.zeros_like(n)); ax[0].set_ylim([-0.05, 1.05]); ax[0].set_title('Input')
l1 = ax[1].plot(n, np.zeros_like(n)); ax[1].set_ylim([-0.2, 1.2]); ax[1].set_title('Output')

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)
    l0[0].set_ydata(x);
    y = scipy.signal.lfilter(b, a, x)
    #y = scipy.signal.filtfilt(b, a, x)
    l1[0].set_ydata(y);

anim = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=True); 
#plt.close(); HTML(anim.to_html5_video())

## Respuesta en frecuencia de filtros FIR e IIR del orden equivalente

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, 2, figsize=(9, 6))
b, a = scipy.signal.iirfilter(N=20, Wn=0.2, ftype='butter', btype='lowpass', output='ba')
h = scipy.signal.firwin(numtaps=20, cutoff=0.2, pass_zero=True, window='hann', fs=2)
freq_fir, response_fir = scipy.signal.freqz(h, 1)
freq_iir, response_iir = scipy.signal.freqz(b, a)

ax[0, 0].set_title('Espectro\nde magnitud'); ax[0, 1].set_title('Retardo')
ax[0, 0].plot(freq_fir/np.pi, np.abs(response_fir))
ax[1, 0].plot(freq_iir/np.pi, np.abs(response_iir)); 
ax[0, 0].set_ylabel('FIR'); ax[1, 0].set_ylabel('IIR'); 
freq_fir, delay_fir = scipy.signal.group_delay(system=(h, 1))
ax[0, 1].plot(freq_fir/np.pi, delay_fir); ax[0, 1].set_ylim([np.mean(delay_fir)-1, np.mean(delay_fir)+1])
freq_iir, delay_iir = scipy.signal.group_delay(system=(b, a))
ax[1, 1].plot(freq_iir/np.pi, delay_iir); 

# Efectos de audio: <a href="https://en.wikipedia.org/wiki/Wah-wah_(music)">Wah-wah</a> 

- Filtro pasabanda modulado
- Ancho de pasada fijo $f_b$ [Hz]
- Frecuencia central variable $f_c$ [Hz]
- La frecuencia central se modula con una onda lenta
- Se modela como el siguiente sistema **IIR**
$$
H[k] = \frac{(1+c)W_N^{2k} -(1+c) }{W_N^{2k} + d(1-c)W_N^k -c}
$$
donde $d=-\cos(2\pi f_c/f_s)$ y $c = \frac{\tan(\pi f_b/f_s) -1}{\tan(2\pi f_b /f_s)+1}$



Ref: https://www.ee.columbia.edu/~ronw/adst-spring2010/lectures/lecture2.pdf

In [ ]:
data, fs = sf.read("data/nomekop.ogg")
Audio(data, rate=int(fs))

In [ ]:
fb, N, Nw = 200, len(data), 5
c  = (np.tan(np.pi*fb/fs) - 1.)/(np.tan(2*np.pi*fb/fs) +1)
data_wah = []
zi = np.zeros(shape=(2,))
for k in range(int(N/Nw)+1):
    fc = 500 + 2000*(np.cos(2.0*np.pi*k*30./fs) +1)/2
    d = -np.cos(2*np.pi*fc/fs)
    data2, zi = scipy.signal.lfilter([(1+c), 0, -(1+c)], [1, d*(1-c), -c], 
                                     x=data[k*Nw:(k+1)*Nw], zi=zi)
    data_wah.append(data2)
Audio(np.hstack(data_wah), rate=int(fs))

Design of recursive filters
http://www-pagines.fib.upc.es/~pds/Lect06.pdf

[&larr; Volver al índice](#index)

<a id="section5"></a>
***

# Generalizando la DFT: Transformada Z

***

Sea una señal muestrada como una secuencia finita $x[n]$, $n=0, \ldots, N-1$

Su **transformada Z** es 

$$
X[z] = \sum_{n=0}^{N-1} x[n] z^{-n},
$$

donde $z$ es un número complejo. Consideremos $z=r e^{j \frac{2\pi}{N} k}$ entonces

$$
X[z] = \sum_{n=0}^{N-1} (x[n]r^{-n}) e^{-j \frac{2\pi}{N} nk},
$$

es decir que 
- la transformada Z es la DFT de la secuencia $x[n]r^{-n}$ 
- la transformada Z existe si $\sum_n |x[n]r^{-n}| \leq \infty$
- si $r=1$ entonces $z= e^{j \frac{2\pi}{N} k}$, y la transformada Z es equivalente a la DFT
- la DFT es un caso particular de la transformada Z

***

### Propiedades 

Tres propiedades relevantes para nuestros propósitos son
- Linealidad:  $Z\{a x[n] +  b y[n]\} = a X[z] + b Y[z]$
- Convolución: $Z\{x[n] * y[n]\}  =  X[z] Y[z]$
- Traslación: $X[z-m] = X[z]z^{-m}$ 


***

# Función de transferencia de un sistema


El sistema IIR general se define con la ecuación de diferencias

$$
\sum_{m=0}^{M} a[m] y[n-m] = \sum_{l=0}^{L} b[l] x[n-l],
$$

Aplicando transformada Z a ambos lados convertimos la ecuación de diferencias en polinomios

$$
\begin{align}
\sum_{m=0}^{M} a[m] Y[z] z^{-m} &= \sum_{l=0}^{L} b[l] X[z] z^{-l} \nonumber \\
Y[z] \sum_{m=0}^{M} a[m]  z^{-m} &= X[z] \sum_{l=0}^{L} b[l]  z^{-l}, \nonumber 
\end{align}
$$
despejando en la última ecuación tenemos
$$
\frac{Y[z]}{X[z]} = H[z] =   \frac{\sum_{l=0}^{L} b[l] z^{-l} }{ \sum_{m=0}^{M} a[m] z^{-m} } , 
$$
donde $H[z]$ se conoce como **función de transferencia** del sistema y suele expresarse de las siguientes formas equivalentes

$$
H[z] = \frac{\sum_{l=0}^{L} b[l] z^{-l} }{ \sum_{m=0}^{M} a[m] z^{-m} } = K \frac{ \prod_{l=1}^L (z-\beta_l)}{ \prod_{m=1}^M (z-\alpha_m)},
$$

- Forma 1: En términos de los coeficientes $\{a\}_l$ y $\{b\}_m$ 
- Forma 2: En términos de la ganancia $K$ y las raices de los polinomios: los **polos** $\{\alpha\}_l$ y los **ceros** $\{\beta\}_m$ 
    - Un cero es una señal compleja que es atenuada por el sistema
    - Un polo es una señal compleja que es amplificada por el sistema
    - ...

***
La respuesta en frecuencia se obtiene de la función de transferencia haciendo

$$
H[k] = H[z=e^{j \frac{2\pi}{N} k} ]
$$

La función de transferencia es la transformada Z de la respuesta al impulso del sistema

$$
H[z] = \sum_{n=0}^{N-1} h[n] z^{-n}
$$


***

***

**Ejemplo:** Para el sistema IIR que vimos anteriormente

$$
y[n] = (1-\gamma) x[n] + \gamma y[n-1]
$$

Aplicando transformada Z y obtenemos la siguiente función de transferencia

$$
\begin{align}
Y[z] &= (1-\gamma) X[z] + \gamma Y[z] z^{-1} \nonumber \\
H[z] = \frac{Y[z]}{X[z]} &= (1-\gamma) \frac{1}{1 - \gamma z^{-1}} = (1-\gamma) \frac{(z - 0)}{(z - \gamma)} \nonumber
\end{align}
$$

que tiene un polo en $\gamma$, un cero en $0$ y ganancia $1-\gamma$

### Filtro pasa todo (all pass filter)

aplicacion definiendo el angulo

- https://www.dsprelated.com/freebooks/filters/Allpass_Filters.html
- http://www.dspguide.com/ch20/2.htm
- http://www.dspguide.com/ch22.htm
- http://www.dspguide.com/ch33.htm
- https://ccrma.stanford.edu/~nfram/220c/tvfilts.html
- https://sound.eti.pg.gda.pl/student/eim/synteza/adamx/eindex.html
- http://www.ee.ic.ac.uk/hp/staff/dmb/courses/DSPDF/00500_Filters.pdf

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, figsize=(6, 4))
n = np.arange(0, 400, step=1)
x = np.sin(2.0*np.pi*20*n/len(n)) + 0.25*np.random.randn(len(n))
f = fftpack.fftshift(fftpack.fftfreq(d=1, n=len(n)))
def update(L):
    y = np.convolve(x, np.ones(shape=(L,))/L, mode='same')
    ax[0].cla(); ax[0].plot(n, x, 'k.', n, y, 'r-'); 
    X = fftpack.fftshift(fftpack.fft(y*scipy.signal.hamming(len(n))))
    ax[1].cla(); ax[1].plot(f, 2*np.absolute(X)/len(n)); 
interact(update, L=SelectionSlider_nice(options=[1, 2, 3, 4, 5, 10, 15, 20]));